<a id=contents></a>

# Baseline Model building - Conditional Random Field model



[1. ETL and Train Test Split](#ETL)

[2. Modelling with Conditional Random Field](#CRF)

[3. Choice of model architectures](#selection)

[4.1 Model 1](#one)

[4.2 Model 2](#two)

[4.2 Model 3](#three)

[4.2 Model 4](#four)

[4.2 Model 5](#five)

[7. Conclusions and model comparison table](#conc)

In [127]:

import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from matplotlib import cm
import numpy as np

from sklearn_crfsuite import CRF, scorers, metrics
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import string
tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

import warnings
warnings.filterwarnings('ignore')

from scipy import stats as ss
import eli5
#baseline sequential evaluation metrics
from seqeval.metrics import accuracy_score as seq_acc
from seqeval.metrics import classification_report as seq_cr
from seqeval.metrics import f1_score as seq_f1_score

#python package for evaluation in line with 
import nereval


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id=ETL ><a/> 

## 1. ETL of data and Train-Test Split
    
[LINK to table of contents](#contents)

In [116]:
with open('clean_data/crf_train_data.pkl', 'rb') as f:
    crf_features_train = pickle.load(f)
    
with open('clean_data/crf_test_data.pkl', 'rb') as f:
    crf_features_test = pickle.load(f)
    
with open('clean_data/crf_valid_data.pkl', 'rb') as f:
    crf_features_valid = pickle.load(f)
    
with open('clean_data/crf_valid_targets.pkl', 'rb') as f:
    crf_targets_valid = pickle.load(f)
    
with open('clean_data/crf_train_targets.pkl', 'rb') as f:
    crf_targets_train = pickle.load(f)
    
with open('clean_data/crf_test_targets.pkl', 'rb') as f:
    crf_targets_test = pickle.load(f)
    

In [123]:
# a reminder of how our feature data is structured - 7th word of 1st sentence
crf_features_train[0][6]

{'word.lower()': 'london',
 'word.istitle()': 1,
 'len(word)': 6,
 'word.isupper()': 0,
 'word.isdigit()': 0,
 'word.prefix_2': 'Lo',
 'word.suffix_2': 'on',
 'word.prefix_3': 'Lon',
 'word.suffix_3': 'don',
 'word.frequency': 0,
 'word.+1_POS': 'TO',
 'word.-1_POS': 'IN',
 'word.-2_POS': 'VBN',
 'word.BOS': 0,
 'word.same_POS_-1': 0}

In [124]:
# and our target data
crf_targets_train[0][6]

'B-geo'

Before modelling, here's a quick reminder of the meaning of the target variable Tags:
* B - beginning of NE chunk
* I - inside NE chunk
* O - not an NE


* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon



In [134]:
print(f'We have {len(crf_features_train)} sentences in our training data.')
print(f'We have {len(crf_features_valid)} sentences in our validation data.')
print(f'We have {len(crf_features_test)} sentences in our test data.')

We have 1799 sentences in our training data.
We have 600 sentences in our validation data.
We have 600 sentences in our test data.


<a id = 'CRF'></a>

## 2. Baseline modelling with a Conditional Random Field

[LINK to table of contents](#contents)

Sklearn's CRF requires the input data to be a list of lists of dicts. I stored these as pickle files in notebook and loaded them above

In [24]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)

In [128]:
#baseline
%time
crf.fit(crf_features_train, crf_targets_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 16.9 µs


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [90]:
# crf object stores our Tag labels
labels = list(crf.classes_)
labels

['O',
 'B-geo',
 'B-gpe',
 'B-per',
 'I-geo',
 'B-org',
 'I-org',
 'B-tim',
 'B-art',
 'I-art',
 'I-per',
 'I-gpe',
 'I-tim',
 'B-nat',
 'B-eve',
 'I-eve',
 'I-nat']

I am going to use the python library seqeval, which has been specifically designed to work with BIO labels and to help with measuring performance on tasks "[such as named-entity recognition, part-of-speech tagging, semantic role labeling](#https://pypi.org/project/seqeval/)".
The `seqeval` package collapses the B- and I- type of tags into fewer NE tags, as seen in the classification report below. 

In [139]:
crf_y_pred_train = crf.predict(crf_features_train)
seq_f1_score(crf_targets_train, crf_y_pred_train,
                      average='weighted')

0.9039903264812574

### Classification Report Interpretation for train and validation data:

The main reported figure is the weighted average F1 Score:

$$    F_{1} = 2* \frac{Precision * Recall}{Precision + Recall}         $$

The support column refers to how many instances there are of each class. As we've seen before, this distribution is dominated by 'O' (non-NE) and there are some, such as nat ('national phenomena') that are almost zero (there were only 17 instances in the training data). 

The table below gives us the relevant metrics for our baseline model's performance. I would draw your attention to the near bottom right, were we see the macro-average F1 score across all the NE categories. The support column indicates how many instances of each NE there are across the data. 

In [140]:
print(seq_cr(crf_targets_train, crf_y_pred_train, digits=3))

             precision    recall  f1-score   support

        geo      0.849     0.936     0.890      1174
        gpe      0.906     0.858     0.881       808
        org      0.914     0.810     0.859       784
        tim      0.973     0.907     0.939       680
        per      0.972     0.970     0.971       633
        nat      1.000     0.941     0.970        17
        eve      0.958     0.958     0.958        24
        art      0.953     0.891     0.921        46

avg / total      0.913     0.897     0.904      4166



We have plenty of evidence of overfitting, our average F1 score dropping down from 0.9 to 0.7, so we will be fitting a new model using crossvalidation and GridSearchCV. 

In [145]:
crf_y_pred_valid = crf.predict(crf_features_valid)
seq_f1_score(crf_targets_valid, crf_y_pred_valid,
                      average='weighted',)

0.7003058103975536

In [143]:
print(seq_cr(crf_targets_valid, crf_y_pred_valid, digits=3))

             precision    recall  f1-score   support

        geo      0.704     0.744     0.724       422
        gpe      0.762     0.794     0.778       218
        per      0.758     0.684     0.719       206
        tim      0.862     0.720     0.785       243
        art      0.000     0.000     0.000         5
        org      0.532     0.482     0.506       226
        eve      0.500     0.267     0.348        15
        nat      0.000     0.000     0.000         1

avg / total      0.716     0.686     0.699      1336



## 3. Optimisation with GridSearchCV and fine-tuning

In [146]:
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

seq_f1_scorer = make_scorer(seq_f1_score,
                        average='weighted')

crf_params = {'c1': [0.05, 0.1, 0.5, 1.0, 1.5,  2.0],
              'c2': [0.05, 0.1,  0.5, 1.0, 1.5,  2.0]}

grid =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid.fit(crf_features_train, crf_targets_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 14.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                           all_possible_transitions=True, averaging=None,
                           c=None, c1=None, c2=None,
                           calibration_candidates=None, calibration_eta=None,
                           calibration_max_trials=None, calibration_rate=None,
                           calibration_samples=None, delta=None, epsilon=None,
                           error_sensitive=None, gamma=None,
                           keep_tempfi...
                           max_iterations=100, max_linesearch=None,
                           min_freq=None, model_filename=None,
                           num_memories=None, pa_type=None, period=None,
                           trainer_cls=None, variance=None, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'c1': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0],
                         'c2': [0.05, 0.

In [147]:
best_crf = grid.best_estimator_

In [151]:
grid.best_params_

{'c1': 0.1, 'c2': 0.1}

In [162]:
# further fine tuning around the c1 penalty term
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

crf_params = {'c1': [0.01, 0.02, 0.03, 0.04, 0.05,],
              'c2': [ 0.45, 0.5, 0.55]}

grid =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid.fit(crf_features_train, crf_targets_train)

best_crf = grid.best_estimator_

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


TypeError: 'CRF' object is not iterable

In [158]:
grid_2.best_params_

{'c1': 0.15, 'c2': 0.095}

In [159]:
seq_f1_score(crf_targets_train, best_crf_2.predict(crf_features_train))

0.9012823614807646

In [160]:
seq_f1_score(crf_targets_valid, best_crf_2.predict(crf_features_valid))

0.6995412844036697

In [54]:
# further fine tuning around the c1 penalty term
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

crf_params = {'c1': [0.0375, 0.04, 0.0425,],
              'c2': [ 0.425, 0.45, 0.475]}

grid_3 =  GridSearchCV(crf_optim, 
                    crf_params, 
                    f1_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid_3.fit(crf_features_train, crf_targets_train)

best_crf_3 = grid_3.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  4.5min finished


In [60]:
crf_y_valid_pred = best_crf.predict(crf_features_valid)

print(seq_cr(crf_targets_valid, crf_y_valid_pred, digits=3))

              precision    recall  f1-score   support

           O      0.983     0.989     0.986     11139
       B-art      0.000     0.000     0.000         2
       I-art      0.000     0.000     0.000         2
       B-eve      0.500     0.333     0.400         6
       I-eve      0.333     0.250     0.286         4
       B-geo      0.771     0.768     0.770       474
       I-geo      0.568     0.613     0.590        75
       B-gpe      0.822     0.814     0.818       204
       I-gpe      0.000     0.000     0.000         2
       B-nat      0.000     0.000     0.000         2
       I-nat      0.000     0.000     0.000         0
       B-org      0.668     0.612     0.639       227
       I-org      0.747     0.634     0.686       205
       B-per      0.809     0.709     0.755       268
       I-per      0.687     0.908     0.782       239
       B-tim      0.917     0.797     0.853       237
       I-tim      0.828     0.545     0.658        88

   micro avg      0.950   

So we have been able to achieve a weighted average F1 score of **0.949 on the validation data** (0.980 on train data respectively). Therefore there is a small train to test drop in performance, which looks greater when you compare the macro average F1 (test : 0.484; train : 0.850). 

In [73]:
print("Our overall macro-average F1 score is", round(seq_f1_score(crf_targets_valid, crf_y_valid_pred, average='macro'),3))

Our overall macro-average F1 score is 0.729


In [76]:
print("Our overall accuracy is", round(seq_acc(crf_targets_valid, crf_y_valid_pred),4),)


Our overall accuracy is 0.9497


As you'd expect, the results are very different compared to sklearn's estimation, however this is a much more realistic picture of how well our model is performing. The model is pulled down considerably by the low-frequency classes of 'artefacts' and 'natural phenomena'. 


In [69]:
print(seq_cr(crf_targets_valid, crf_y_valid_pred))

             precision    recall  f1-score   support

        geo       0.76      0.76      0.76       474
        gpe       0.82      0.81      0.82       204
        tim       0.85      0.74      0.79       237
        org       0.65      0.59      0.62       227
        per       0.69      0.61      0.65       268
        eve       0.50      0.33      0.40         6
        art       0.00      0.00      0.00         2
        nat       0.00      0.00      0.00         2

avg / total       0.75      0.71      0.73      1420



In [100]:
# And if we check our training classification report:
print(seq_cr(crf_targets_train, crf_y_train_pred))

             precision    recall  f1-score   support

        geo       0.60      0.94      0.73      1174
        gpe       0.69      0.79      0.74       808
        org       0.68      0.74      0.71       784
        tim       0.71      0.86      0.78       680
        per       0.71      0.93      0.80       633
        nat       1.00      0.53      0.69        17
        eve       0.56      0.79      0.66        24
        art       0.91      0.65      0.76        46

avg / total       0.67      0.85      0.75      4166



In [101]:
seqeval_scorer = make_scorer(seq_f1_score)


In [102]:
# further fine tuning around the c1 penalty term
crf_optim = CRF(algorithm='lbfgs',
          max_iterations=100,
          all_possible_transitions=True)

seqeval_scorer = make_scorer(seq_f1_score, average='macro')


crf_params = {'c1': [0.0375, 0.04, 0.0425,],
              'c2': [ 0.425, 0.45, 0.475]}

grid_seqeval =  GridSearchCV(crf_optim, 
                    crf_params, 
                    seqeval_scorer, 
                    -1, cv=5, 
                    return_train_score=True, 
                    verbose=True)

grid_seqeval.fit(crf_features_train, crf_targets_train)

best_crf_seqeval = grid_seqeval.best_estimator_


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.6min finished


In [103]:
crf_y_valid_seqeval_preds = best_crf_seqeval.predict(crf_features_valid)

print(seq_cr(crf_targets_valid, crf_y_valid_seqeval_preds))


             precision    recall  f1-score   support

        geo       0.71      0.76      0.74       422
        gpe       0.81      0.81      0.81       218
        per       0.76      0.72      0.74       206
        tim       0.89      0.72      0.80       243
        art       0.00      0.00      0.00         5
        org       0.56      0.47      0.51       226
        eve       0.50      0.27      0.35        15
        nat       0.00      0.00      0.00         1

avg / total       0.74      0.70      0.71      1336



In [106]:
print("Model has a macro-average F1 score of", round(seq_f1_score(crf_targets_valid, crf_y_valid_seqeval_preds, average='macro'),3))

Model has a macro-average F1 score of 0.717


In [107]:
print("Our overall accuracy is", round(seq_acc(crf_targets_valid, crf_y_valid_seqeval_preds),3),)


Our overall accuracy is 0.951


<a id=ttsplit ><a/> 

## 3. Investigating our best model's weights
   
[LINK to table of contents](#contents)

In [114]:

fig = eli5.show_weights(best_crf_seqeval, top=30)
fig

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.684,1.245,-1.359,1.356,-1.194,1.63,-1.962,1.314,-1.014,0.883,-1.016,1.552,-3.008,2.001,-1.907,1.958,-2.629
B-art,-0.135,-0.031,3.472,-0.003,-0.097,-0.439,-0.338,-0.558,-0.106,0.0,-0.058,-0.4,-0.631,-0.482,-0.528,0.02,-0.302
I-art,-0.628,-0.04,3.439,0.0,-0.06,-0.301,-0.181,-0.224,-0.052,0.0,-0.021,-0.254,-0.352,-0.571,-0.546,0.003,-0.12
B-eve,-0.748,-0.013,-0.101,-0.053,3.637,-0.36,-0.293,-0.412,-0.093,0.0,-0.061,-0.303,-0.616,-0.367,-0.388,-0.267,-0.18
I-eve,-0.066,0.0,-0.032,-0.26,2.253,-0.113,-0.099,-0.112,-0.033,0.0,0.0,-0.174,-0.213,-0.139,-0.334,-0.11,-0.055
B-geo,0.697,-0.303,-0.734,-0.183,-0.533,-1.239,3.864,0.022,-0.775,-0.102,-0.41,-1.281,-1.584,-1.532,-1.493,1.076,-1.116
I-geo,0.222,-0.118,-0.206,-0.012,-0.1,-0.851,3.142,-0.802,-0.256,-0.02,-0.105,-0.462,-0.905,-0.741,-0.834,0.279,-0.524
B-gpe,0.904,-0.339,-0.594,-0.233,-0.7,-1.104,-1.176,-1.603,2.997,-0.135,-0.384,1.08,-1.805,0.751,-1.236,-1.131,-1.117
I-gpe,-0.168,0.0,-0.024,0.0,0.0,0.27,-0.089,-0.188,2.647,0.0,0.0,-0.195,-0.307,-0.37,-0.362,-0.084,-0.058
B-nat,-0.345,0.0,-0.042,0.0,-0.061,-0.24,-0.107,-0.233,-0.017,-0.028,2.497,-0.225,-0.304,-0.27,-0.304,-0.161,-0.094


<a id=selection></a>

## 4. Choice of model architectures

[LINK to table of contents](#contents)

<a id=one ><a/> 

## 4.1 Model 1
    
[LINK to table of contents](#contents)

<a id=two ><a/> 

## 4.2 Model 2
    
[LINK to table of contents](#contents)

<a id=three ><a/> 

## 4.3 Model 3
    
[LINK to table of contents](#contents)

<a id=four ><a/> 

## 4.4 Model 4
    
[LINK to table of contents](#contents)

<a id=five ><a/> 

## 4.5 Model 5
    
[LINK to table of contents](#contents)

<a id=conc ><a/> 

## 7. Conclusions and model comparison table
    
[LINK to table of contents](#contents)